In [1]:
"""  Import necessary packages """

import rioxarray as rxr
import LoadSaveFunctions as lsf
import matplotlib.pyplot as plt
import numpy as np
import ipyleaflet
import ipywidgets as ipw
from pyproj import CRS, Transformer
from matplotlib.ticker import FixedLocator

In [ ]:
""" select file to open with its path """

file_path = lsf.load_file()

In [ ]:
""" Load and check raster file """

try:
    data = rxr.open_rasterio(file_path, masked=True)
    print("Geotiff file loaded successfully.")
    print(data)
except Exception as e:
    print(f"Error loading the Geotiff file: {e}")

# View generate metadata associated with the raster file
print("The crs of your data is:", data.rio.crs)
print("The nodatavalue of your data is:", data.rio.nodata)
print("The shape of your data is:", data.shape)
print("The spatial resolution for your data is:", data.rio.resolution())
print("The metadata for your data is:", data.attrs)
print("The spatial extent of this data is: ", data.rio.bounds())

In [ ]:
""" Plot histogram of values """

f, ax = plt.subplots(figsize=(8, 5))
data.plot.hist(ax=ax, color="purple", bins=30, edgecolor='white')
ax.set(title="Distribution of stable isotope values",
       xlabel='Stable isotope compositions (‰ V-SMOW)',
       ylabel='Frequency')
plt.show()

print("The minimum raster value is: ", np.nanmin(data.data))
print("The maximum raster value is: ", np.nanmax(data.data))

In [ ]:
""" Plot a limited area of the raster """

# Définir la projection de ton GeoTIFF (ESRI:54012) et WGS 84 (latitude/longitude)
crs_mollweide = CRS.from_string('ESRI:54012')  # CRS pour Mollweide
crs_wgs84 = CRS.from_epsg(4326)  # CRS pour WGS 84 (EPSG:4326)

# Créer un Transformer pour convertir les coordonnées
transformer_to_wgs84 = Transformer.from_crs(crs_mollweide, crs_wgs84, always_xy=True)

# Limites géographiques de la France en WGS 84 (longitude, latitude)
lon_min, lon_max = -5.14, 9.56
lat_min, lat_max = 41.33, 51.09

# Convertir les coordonnées WGS 84 (lon/lat) vers Mollweide (x/y)
transformer_to_mollweide = Transformer.from_crs(crs_wgs84, crs_mollweide, always_xy=True)
x_min, y_min = transformer_to_mollweide.transform(lon_min, lat_min)
x_max, y_max = transformer_to_mollweide.transform(lon_max, lat_max)

# Filtrer les données du GeoTIFF en fonction des coordonnées projetées
subset = data.sel(x=slice(x_min, x_max), y=slice(y_max, y_min))

# Créer un plot
fig, ax = plt.subplots(figsize=(10, 8))
im = ax.imshow(subset[0], origin='upper', extent=[x_min, x_max, y_min, y_max], cmap='viridis')

# Définir les ticks (graduations) et convertir en latitude/longitude
def update_ticks(ax, transformer, axis='x'):
    # Récupérer les valeurs actuelles des ticks
    ticks = ax.get_xticks() if axis == 'x' else ax.get_yticks()

    # Transformer les coordonnées en lat/lon
    new_labels = []
    for tick in ticks:
        lon, lat = transformer.transform(tick, 0) if axis == 'x' else transformer.transform(0, tick)
        new_labels.append(lon if axis == 'x' else lat)

    # Fixer les emplacements des ticks et leurs étiquettes
    if axis == 'x':
        ax.xaxis.set_major_locator(FixedLocator(ticks))
        ax.set_xticklabels([f"{lon:.2f}" for lon in new_labels])
    else:
        ax.yaxis.set_major_locator(FixedLocator(ticks))
        ax.set_yticklabels([f"{lat:.2f}" for lat in new_labels])

# Mettre à jour les axes x et y avec les nouvelles coordonnées
update_ticks(ax, transformer_to_wgs84, axis='x')  # Pour l'axe des x (longitude)
update_ticks(ax, transformer_to_wgs84, axis='y')  # Pour l'axe des y (latitude)

# Ajouter des étiquettes aux axes
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Afficher la barre de couleur et la figure
plt.colorbar(im, ax=ax)
plt.title('Portion du GeoTIFF centrée sur la France avec axes en lat/lon')
plt.show()


In [ ]:
""" Create a map from openstreetmap centered on given coordinates (latitude, longitude) that we can browse and click to get coordinates """

# Defining global variables to store latitude, longitude and marker state
lat = None
lon = None
current_marker = None

# Create map centered on given coordiantes
center = [45.7586, 4.8333]  # Lyon
mymap = ipyleaflet.Map(basemap=ipyleaflet.basemaps.Esri.WorldStreetMap, center=center, zoom=3, layout=ipw.Layout(width='1000px', height='600px'), scroll_wheel_zoom = True)


# HTML widget to display coordinates
coord_label = ipw.HTML("Click on map to get coordiantes")
coord_label.value = "Coordinates : "

# Function to get coordinates by clicking on the map
def on_map_interaction(**kwargs):
    global lat, lon, current_marker
    if kwargs.get('type') == 'click':
        lat, lon = kwargs.get('coordinates')
        coord_label.value = f"Coordonnées : Latitude {lat}, Longitude {lon}"

        # Remove existing current marker
        if current_marker is not None:
            mymap.remove_layer(current_marker)
        
        # Create new marker on click position
        current_marker = ipyleaflet.Marker(location=(lat, lon))
        mymap.add_layer(current_marker)

# link interaction event with function
mymap.on_interaction(on_map_interaction)

# Display map and widget
ui = ipw.VBox([coord_label, mymap])
ui


